In [222]:
import random
import numpy as np
import PDSVRP_instance
import copy

In [223]:
def get_truck_customers(truck_tours):
    truck_customers = []
    for tour in truck_tours:
        for customer in tour:
            truck_customers.append(customer)

    return truck_customers

In [224]:
def get_drone_customers(drone_tours):
    drone_customers = []
    for tour in drone_tours:
        for customer in tour:
            drone_customers.append(customer)

    return drone_customers

In [225]:
def truck_tour_time(truck_travel_times, tour):
        
    if (len(tour) == 0):
        return 0
    total_time = truck_travel_times[0][tour[0]]
        
    for i in range(len(tour) - 1):

        total_time += truck_travel_times[tour[i]][tour[i + 1]]

           
    total_time += truck_travel_times[tour[-1]][0]
    
    return total_time


In [226]:
def drone_tour_time(drone_travel_times, tour):
   
    if (len(tour) == 0):
        return 0
    
    total_time = 0
    
    for i in range(len(tour)):
        total_time += (drone_travel_times[tour[i]]) * 2

   
    
    return total_time

In [227]:
def total_completion_time(instance, solution):
    total_time = 0
    
    for truck_tour in solution[0][0]:
        total_time += truck_tour_time(instance.t_t, truck_tour)
        
    for drone_tour in solution[0][1]:
        total_time += drone_tour_time(instance.t_d, drone_tour)
        
    return total_time

In [228]:
def is_truck_tour_feasible(instance, tour):
    return (truck_tour_time(instance.t_t, tour) <= instance.T_t) and sum(instance.w[customer] for customer in tour) <= instance.Q_t

In [229]:
def is_drone_tour_feasible(instance, tour):
    for customer in tour:
       if instance.w[customer] > instance.Q_d:
        return False 
    return (drone_tour_time(instance.t_d, tour) <= instance.T_d)

In [230]:
def is_swap_possible(instance, tours, customer, position):
    new_tour = copy.deepcopy(tours[position[0]][position[1]])
    new_tour[position[2]] = customer
    if (position[0] == 0):
        return is_truck_tour_feasible(instance, new_tour)
    if(position[0] == 1):
        return is_drone_tour_feasible(instance, new_tour)
    
    return False

In [231]:
def sort_absent_customers(instance, solution, w1, w2, w3, w4, w5):
    
    absent_customers = solution[1]

    def random_sort(instance, solution, absent_customers):
        random.shuffle(absent_customers)
        return absent_customers
        
    def near_dp(instance, solution, absent_customers):
        absent_customers = sorted(absent_customers, key = lambda customer : instance.distances[0][customer])
        return absent_customers
        
    def far_dp(instance, solution, absent_customers):
        absent_customers = sorted(absent_customers, key = lambda customer : (1/instance.distances[0][customer]))
        return absent_customers
        
    def near_tr(instance, solution, absent_customers):
        truck_customers = get_truck_customers(solution[0][0])
        if (len(truck_customers) == 0):
            return random_sort(instance, solution, absent_customers)
        absent_customers = sorted(absent_customers, key = lambda customer : min(instance.distances[t_customer][customer] for t_customer in truck_customers))
        return absent_customers
        
        
    def far_tr(instance, solution, absent_customers):
        truck_customers = get_truck_customers(solution[0][0])
        if (len(truck_customers) == 0):
            return random_sort(instance, solution, absent_customers)
        absent_customers = sorted(absent_customers, key = lambda customer : 1/(1+(min(instance.distances[t_customer][customer] for t_customer in truck_customers))))
        return absent_customers

    # Lista dei blocchi e dei relativi pesi
    sorting_methods = [random_sort, near_dp, far_dp, near_tr, far_tr]
    weights = [w1, w2, w3, w4, w5]

    selected_method = random.choices(sorting_methods, weights, k=1)[0]

    return selected_method(instance, solution, absent_customers)

In [232]:
def feasible_truck_tour_positions_calculation(instance, truck_tours, customer): #in non-empty truck tours
    feasible_positions = []
    for i in range(len(truck_tours)):
        #if len(truck_tours[i]) > 0:
        for j in range (len(truck_tours[i]) + 1): # lo posso aggiungere anche come ultimo
            new_tour = copy.deepcopy(truck_tours[i])
            new_tour.insert(j, customer)
            if (is_truck_tour_feasible(instance, new_tour)):
                feasible_positions.append([0, i, j])

    return feasible_positions


In [233]:
def insert_customer(customer, pos, solution_):
    
    solution_[0][pos[0]][pos[1]].insert(pos[2], customer)
    
    return solution_

In [234]:
def is_drone_eligible(instance, drone_tours, customer):
    if instance.w[customer] > instance.Q_d:
        return False
    for tour in drone_tours:
        new_tour = copy.deepcopy(tour)
        new_tour.append(customer)
        if (drone_tour_time(instance.t_d, new_tour) <= instance.T_d):
            return True
    
    return False

In [235]:
def largest_spatial_slack_drone(instance, drone_tours):
    chosen_drone = min(range(len(drone_tours)), key=lambda i: drone_tour_time(instance.t_d, drone_tours[i]))
    return chosen_drone

In [236]:
def remove_drone_customer(drone_tour, absent_customers, customer_to_remove):
    new_tour = drone_tour
    new_tour.remove(customer_to_remove)
    new_absent_vector = absent_customers + [customer_to_remove]

    return new_tour, new_absent_vector

#test ok

In [237]:
def get_position(customer, tours):
    for vehicle_index in range(len(tours)): 
        for tour_index in range(len(tours[vehicle_index])):
            for customer_index in range(len(tours[vehicle_index][tour_index])):
                if tours[vehicle_index][tour_index][customer_index] == customer:
                    return [vehicle_index, tour_index, customer_index]
                
    return None

In [238]:
def select_nearest_neighbors(n_nearest, distances_from_customer):
    indexed_array = [(value, idx) for idx, value in enumerate(distances_from_customer) if idx != 0]
    
    indexed_array.sort(key=lambda x: x[0])
    
    nearest_neighbors = [idx for value, idx in indexed_array[1:n_nearest + 1]] #the depot and the customer itself are escluded
    
    return nearest_neighbors

#test ok

In [239]:
def find_customer_in_vehicle_tours(tours, customer):
    for i, row in enumerate(tours):
        for j, element in enumerate(row):
            if element == customer:
                return [i, j]
    return None

In [282]:
def _2_opt_x(instance, solution, customer, neighbor, customer_tour_index):
    neighbor_index = find_customer_in_vehicle_tours(solution[0][0], neighbor)
    customer_index = [customer_tour_index, next((index for index, c in enumerate(solution[0][0][customer_tour_index]) if c == customer), -1)]
    new_tour_1 = solution[0][0][neighbor_index[0]][:neighbor_index[1]] + solution[0][0][customer_index[0]][customer_index[1]:]
    new_tour_2 = solution[0][0][customer_index[0]][:customer_index[1]] + solution[0][0][neighbor_index[0]][neighbor_index[1]:]
    if (is_truck_tour_feasible(instance, new_tour_1) and is_truck_tour_feasible(instance,new_tour_2)):
        solution[0][0][neighbor_index[0]] = new_tour_1
        solution[0][0][customer_index[0]] = new_tour_2
        
    return solution

In [270]:
def relocate(instance, solution, customer, neighbor, tour_index):
    #customer_index = next((index for index, c in enumerate(solution[0][0][tour_index]) if c == customer), -1)
    neighbor_index = next((index for index, c in enumerate(solution[0][0][tour_index]) if c == neighbor), -1)

    proposed_tour = copy.deepcopy(solution[0][0][tour_index])
    proposed_tour.remove(customer)
    proposed_tour.insert(neighbor_index, customer)

    other_tour = copy.deepcopy(solution[0][0][tour_index])
    other_tour.remove(customer)
    other_tour.insert(neighbor_index + 1, customer)

    if (truck_tour_time(instance.t_t, proposed_tour) > truck_tour_time(instance.t_t, other_tour)):
        proposed_tour = other_tour

    if (is_truck_tour_feasible(instance, proposed_tour)):
        solution[0][0][tour_index] = proposed_tour
        
    return solution




In [242]:
def swap(instance, solution, customer, neighbor, tour_index):
    customer_index = next((index for index, c in enumerate(solution[0][0][tour_index]) if c == customer), -1)
    neighbor_index = next((index for index, c in enumerate(solution[0][0][tour_index]) if c == neighbor), -1)

    proposed_tour = copy.deepcopy(solution[0][0][tour_index])
    proposed_tour[customer_index] = neighbor
    proposed_tour[neighbor_index] = customer

    if (is_truck_tour_feasible(instance, proposed_tour)):
        solution[0][0][tour_index] = proposed_tour

    return solution

In [243]:
def _2_opt(instance, solution, customer, neighbor, tour_index):
    customer_index = next((index for index, c in enumerate(solution[0][0][tour_index]) if c == customer), -1)
    neighbor_index = next((index for index, c in enumerate(solution[0][0][tour_index]) if c == neighbor), -1)

    i = min(customer_index, neighbor_index)
    j = max(customer_index, neighbor_index)

    proposed_tour = copy.deepcopy(solution[0][0][tour_index])

    if (i == 0):
        proposed_tour[:j+1] = proposed_tour[j::-1]
        return solution

    

    proposed_tour[i:j+1] = proposed_tour[j:i-1:-1]

    if (is_truck_tour_feasible(instance, proposed_tour)):
        solution[0][0][tour_index] = proposed_tour

    return solution

#test ok

In [244]:
def swap_x(instance, solution, customer, neighbor, customer_tour_index):
    customer_index = [customer_tour_index, next((index for index, c in enumerate(solution[0][0][customer_tour_index]) if c == customer), -1)]
    neighbor_index = find_customer_in_vehicle_tours(solution[0][1], neighbor)


    modified_truck_tour = copy.deepcopy(solution[0][0][customer_tour_index])
    modified_truck_tour[customer_index[1]] = neighbor

    modified_drone_tour = copy.deepcopy(solution[0][1][neighbor_index[0]])
    modified_drone_tour[neighbor_index[1]] = customer

    if(is_truck_tour_feasible(instance, modified_truck_tour) and is_drone_tour_feasible(instance, modified_drone_tour)):
        solution[0][0][customer_index[0]] = modified_truck_tour
        solution[0][1][neighbor_index[0]] = modified_drone_tour

    return solution

In [245]:
def shift_t(instance, solution, customer, customer_tour_index):
    if (is_drone_eligible(instance, solution[0][1], customer)):
        solution[0][0][customer_tour_index].remove(customer)
        chosen_drone = largest_spatial_slack_drone(instance, solution[0][1])
        solution[0][1][chosen_drone].append(customer)

    return solution


In [246]:
def shift_d(instance, solution, customer, customer_tour_index): #sistemare il shift_d
    new_solution = copy.deepcopy(solution)
    new_solution[0][1][customer_tour_index].remove(customer)

    pos_best = None
    feasible_truck_tour_positions = feasible_truck_tour_positions_calculation(instance, solution[0][0], customer)
    for pos in feasible_truck_tour_positions: 
        if pos_best == None or  (pos_best != None and total_completion_time(instance, insert_customer(customer, pos, copy.deepcopy(solution))) < total_completion_time(instance, insert_customer(customer, pos_best, copy.deepcopy(solution)))):
            pos_best = pos

    if pos_best == None:
        return solution, pos_best

    solution = insert_customer(customer, pos_best, new_solution)
    return solution

In [247]:
def sort_by_angular(coordinates, c_drones, grad_seed):

    rad_seed = np.deg2rad(grad_seed)
    ref_point = coordinates[0]

    def polar_angle(coord):
        # Compute the angle in radians with respect to the reference point
        dx = coord[0] - ref_point[0]
        dy = coord[1] - ref_point[1]
        angle = np.arctan2(dy, dx)
        return angle

    def angular_distance(angle1, angle2):
        # Compute the absolute angular distance between two angles
        return min(abs(angle1 - angle2), 2 * np.pi - abs(angle1 - angle2))

    # Calculate the polar angles for all nodes in c_drones
    angles = [polar_angle(coordinates[i]) for i in c_drones]

    # Calculate the angular distance from grad_seed
    distances = [angular_distance(angle, rad_seed) for angle in angles]

    # Pair the indices with their angular distances
    indexed_distances = list(zip(c_drones, distances))

    # Sort based on angular distance
    indexed_distances.sort(key=lambda x: x[1])

    # Extract the sorted indices
    sorted_c_drones = [index for index, _ in indexed_distances]

    return sorted_c_drones

#test ok

In [248]:
def sweep_removal_operator(instance, solution, sigma):
    c_drones = get_drone_customers(solution[0][1])
    nbSweept = random.randint(0,int(len(c_drones) * sigma))

    grad_seed = random.randint(0, 359)
    c_drones = sort_by_angular(instance.coordinates, c_drones, grad_seed)
    for c in c_drones:
        if (len(solution[1]) < nbSweept):

            #inlcude tour_index calculation inside "remove_drone_customer, params: solution[0][1], c"
            tour_index = next((index for index, drone_tour in enumerate(solution[0][1]) if c in drone_tour), -1)
            solution[0][1][tour_index], solution[1] = remove_drone_customer(solution[0][1][tour_index], solution[1], c)
                               

    return solution

In [249]:
def random_drone_customer_removal(solution, sigma):
    #print("solution befor rand remove drones", solution)
    c_drones = get_drone_customers(solution[0][1])
    q = random.randint(0,int(len(c_drones) * sigma))
    
    
    while (len(solution[1]) < q):
        customer_to_remove = random.choice(c_drones)

        c_drones.remove(customer_to_remove)
        tour_index = next((index for index, drone_tour in enumerate(solution[0][1]) if customer_to_remove in drone_tour), -1)
        solution[0][1][tour_index], solution[1] = remove_drone_customer(solution[0][1][tour_index], solution[1], customer_to_remove)

    return solution

In [250]:
def max_string_length(truck_tours, L_max):
    n_non_empty_truck_tours = 0
    for truck_tour in truck_tours:
        if len(truck_tour) > 0:
            n_non_empty_truck_tours += 1

    average_tour_cardinality = len(get_truck_customers(truck_tours)) / (n_non_empty_truck_tours)
    return min(average_tour_cardinality, L_max)

In [251]:
def n_strings_to_remove(c_average_removed, l_s_max):
    k_s_max = (4 * c_average_removed) / (1 + l_s_max) - 1
    k_s = int(random.uniform(1, k_s_max + 1))  
    
    return k_s

# test ok

In [252]:
def sort_by_euclidean_distance(distance_matrix, c_seed):
    distances = [distance_matrix[c_seed][i] for i in range(1, len(distance_matrix[c_seed]))]

    sorted_c = np.argsort(distances)
    sorted_c = [x + 1 for x in sorted_c]

    return sorted_c

# test ok

In [253]:
def string_to_remove_length(l_s_max, truck_tour):
    l_t_max = min(len(truck_tour), l_s_max)
    l_t = int(random.uniform(1, l_t_max + 1))
    return l_t

# test ok

In [254]:
def remove_string(truck_tour, string_length, customer, absent_customers):
    customer_index = next((index for index, c in enumerate(truck_tour) if c == customer), -1)
    if (len(truck_tour) < customer_index + string_length):
        absent_customers += truck_tour[len(truck_tour)-string_length:]
        truck_tour = truck_tour[:len(truck_tour)-string_length]

    else:
        absent_customers += truck_tour[customer_index: (customer_index + string_length)]
        truck_tour = truck_tour[:customer_index] + truck_tour[customer_index + string_length:]

    return truck_tour, absent_customers

In [255]:
def string_removal(instance, solution, c_average_removed, L_max): 
    l_s_max = max_string_length(solution[0][0], L_max)
    k_s = n_strings_to_remove(c_average_removed, l_s_max)
    
    c_seed = random.randint(1, instance.N - 1)

    R = []

    c_adj = sort_by_euclidean_distance(instance.distances, c_seed)

    for c in c_adj:
        if len(R) < k_s:
            if any(c in truck_tour for truck_tour in solution[0][0]):
                if c not in solution[1]:
                    tour_index = next((index for index, truck_tour in enumerate(solution[0][0]) if c in truck_tour), -1)
                    if tour_index not in R:
                        l = string_to_remove_length(l_s_max, solution[0][0][tour_index])
                        
                        solution[0][0][tour_index], solution[1] = remove_string(solution[0][0][tour_index], l, c, solution[1])
                        R.append(tour_index)
            elif any(c in drone_tour for drone_tour in solution[0][1]):
                if c not in solution[1]:
                    tour_index = next((index for index, drone_tour in enumerate(solution[0][1]) if c in drone_tour), -1)
                    solution[0][1][tour_index], solution[1] = remove_drone_customer(solution[0][1][tour_index], solution[1], c)


    return solution

In [256]:
def makespan(instance, solution):
    slowest_tour_time = 0
    
    for truck_tour in solution[0][0]:
        curr_time_tour = truck_tour_time(instance.t_t, truck_tour)
        slowest_tour_time = curr_time_tour if curr_time_tour >= slowest_tour_time else slowest_tour_time
        
    for drone_tour in solution[0][1]:
        curr_time_tour = drone_tour_time(instance.t_d, drone_tour)
        slowest_tour_time = curr_time_tour if curr_time_tour >= slowest_tour_time else slowest_tour_time
        
    return slowest_tour_time

##### First modification

In [360]:
def recreate_min_time(instance, solution, w1, w2, w3, w4, w5, gamma):
    solution[1] = sort_absent_customers(instance, solution, w1, w2, w3, w4, w5)
    while len(solution[1]) > 0:
        c = solution[1][0]
        pos_best = None
        makespan_best = float('inf')
        
        feasible_truck_tour_positions = feasible_truck_tour_positions_calculation(instance, solution[0][0], c)
        print("feasible_truck_tour_positions: ", feasible_truck_tour_positions)
        for pos in feasible_truck_tour_positions: 
            makespan_diff = truck_tour_time(instance.t_t, insert_customer(c, pos, copy.deepcopy(solution))[0][pos[0]][pos[1]]) - makespan(instance, copy.deepcopy(solution))
            print("makespan: ", makespan(instance, copy.deepcopy(solution)))
            print("makespan_diff_trucks: ", makespan_diff)
            print("pos: ", pos)
            random_v = random.random()
            print("random value: ", random_v)

            if pos_best == None or (pos_best != None and makespan_diff <= makespan_best and makespan_diff >= 0 and random_v > (1 - gamma)):
                print("I got here")
                makespan_best = makespan_diff
                pos_best = pos
        
        if is_drone_eligible(instance, solution[0][1], c):
            chosen_drone = largest_spatial_slack_drone(instance, solution[0][1]) 
            makespan_diff = drone_tour_time(instance.t_d, insert_customer(c, [1, chosen_drone, 0], copy.deepcopy(solution))[0][1][chosen_drone]) - makespan(instance, copy.deepcopy(solution))
            print("makespan_diff_drones: ", makespan_diff)
            print(makespan_diff <= makespan_best)  
            if pos_best == None or (pos_best != None and makespan_diff <= makespan_best and makespan_diff >= 0 and random.random() > (1 - gamma)):
                makespan_best = makespan_diff
                pos_best = [1, chosen_drone, 0]

        if pos_best == None:
            empty_truck_tour_index = next((index for index, truck_tour in enumerate(solution[0][0]) if len(truck_tour) == 0), -1)
            if empty_truck_tour_index == -1:
                return solution
            
            pos_best = [0, empty_truck_tour_index, 0]
        
        print("pos_best: ", pos_best)
        print("customer: ", c)
        solution = insert_customer(c, pos_best, solution)
        solution[1].remove(c)
        print("solution: ", solution)
        print("solution[1]: ", solution[1])

    return solution

In [258]:
def ruin_and_recreate(instance, solution, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma):
    r = random.random()
    if (r < 0.5):
        solution = sweep_removal_operator(instance, solution, sigma)
    else:
        solution = random_drone_customer_removal(solution, sigma)

    solution = string_removal(instance, solution, c_average_removed, L_max)

    solution = recreate_min_time(instance, solution, w1, w2, w3, w4, w5, gamma)

    return solution


In [259]:
def perturbate(instance, solution, p_min, p_max, max_unfeasible_swaps):
    p = random.randint(p_min, p_max)
    swaps_executed = 0
    unfeasible_swaps = 0
    while (swaps_executed < p and unfeasible_swaps < max_unfeasible_swaps):
        c1 = random.randint(1, instance.N - 1)
        c2 = random.randint(1, instance.N - 1)
        pos1 = get_position(c1, solution[0])
        pos2 = get_position(c2, solution[0])
        if (is_swap_possible(instance, solution[0], c1, pos2) and is_swap_possible(instance, solution[0], c2, pos1)):
            solution[0][pos1[0]][pos1[1]][pos1[2]] = c2
            solution[0][pos2[0]][pos2[1]][pos2[2]] = c1
            swaps_executed += 1
            unfeasible_swaps = 0
        else:
            unfeasible_swaps += 1

    return solution

##### Second modification

In [285]:
def local_search_min_time(instance, solution, n_nearest):
    print(solution)

    for i in range(len(solution[0][0])):

        for customer in solution[0][0][i]:

            neighbors = select_nearest_neighbors(n_nearest, instance.distances[customer])
            print("neighbors: ", neighbors)
            for neighbor in neighbors:
                neighbor_pos = get_position(neighbor, copy.deepcopy(solution[0]))

                if (neighbor in get_truck_customers(solution[0][0])):
                    neighbor_index = next((j for j, node in enumerate(solution[0][0][i]) if node == neighbor), None)
                    
                    if neighbor_index == None:
                        t_1 = truck_tour_time(instance.t_t, copy.deepcopy(solution[0][0][i]))
                        t_2 = truck_tour_time(instance.t_t, copy.deepcopy(solution[0][neighbor_pos[0]][neighbor_pos[1]]))
                        t = max(t_1, t_2)
                        
                        print("customer: ", customer)
                        print("neighbor: ", neighbor)
                        
                        print("solution[0][0][i]: ", solution[0][0][i])
                        print("solution[0][neighbor_pos[0]][neighbor_pos[1]]: ", solution[0][neighbor_pos[0]][neighbor_pos[1]])

                        new_solution = _2_opt_x(instance, copy.deepcopy(solution), customer, neighbor, i)
                        
                        print(new_solution)
                        t_1_x = truck_tour_time(instance.t_t, copy.deepcopy(new_solution[0][0][i]))
                        t_2_x = truck_tour_time(instance.t_t, copy.deepcopy(new_solution[0][neighbor_pos[0]][neighbor_pos[1]]))
                        
                        print("new_solution[0][0][i]: ", new_solution[0][0][i])
                        print("new_solution[0][neighbor_pos[0]][neighbor_pos[1]]: ", new_solution[0][neighbor_pos[0]][neighbor_pos[1]])
                        
                        print("t: ", t)
                        print("t_1_x: ", t_1_x)
                        print("t_2_x: ", t_2_x)
                        if ((t_1_x <= t and t_2_x < t) or (t_1_x < t and t_2_x <= t)):
                            return local_search_min_time(instance, copy.deepcopy(new_solution), n_nearest)

                    else: 
                        new_solution = relocate(instance, copy.deepcopy(solution), customer, neighbor, i)
                        if total_completion_time(instance, copy.deepcopy(new_solution)) < total_completion_time(instance, copy.deepcopy(solution)):
                            return local_search_min_time(instance, new_solution, n_nearest)

                        new_solution = swap(instance, copy.deepcopy(solution), customer, neighbor, i)
                        if total_completion_time(instance, copy.deepcopy(new_solution)) < total_completion_time(instance, copy.deepcopy(solution)):
                            return local_search_min_time(instance, new_solution, n_nearest)

                        new_solution = _2_opt(instance, copy.deepcopy(solution), customer, neighbor, i)
                        if total_completion_time(instance, copy.deepcopy(new_solution)) < total_completion_time(instance, copy.deepcopy(solution)):
                            return local_search_min_time(instance, new_solution, n_nearest)

                else:
                    t_1 = truck_tour_time(instance.t_t, copy.deepcopy(solution[0][0][i]))
                    t_2 = drone_tour_time(instance.t_d, copy.deepcopy(solution[0][neighbor_pos[0]][neighbor_pos[1]]))
                    t = max(t_1, t_2)

                    new_solution = swap_x(instance, copy.deepcopy(solution), customer, neighbor, i)
                    t_1_x = truck_tour_time(instance.t_t, copy.deepcopy(new_solution[0][0][i]))
                    t_2_x = drone_tour_time(instance.t_d, copy.deepcopy(new_solution[0][neighbor_pos[0]][neighbor_pos[1]]))

                    if ((t_1_x <= t and t_2_x < t) or (t_1_x < t and t_2_x <= t)):
                        return local_search_min_time(instance, new_solution, n_nearest)


            t_1 = truck_tour_time(instance.t_t, copy.deepcopy(solution[0][0][i]))
            t_2 = 0 # drone_tour_time(instance.t_d, solution[0][1][chosen_drone]) = 0 perchè il customer non è ancora stato assengato a chosen_drone
            t = max(t_1, t_2)

            new_solution = shift_t(instance, copy.deepcopy(solution), customer, i)
            t_1_x = 0
            new_customer_pos = get_position(customer, copy.deepcopy(solution[0])) #new_customer_pos contiene la pos del customer che è stato spostato dal truck tour al drone tour
            t_2_x = drone_tour_time(instance.t_d, copy.deepcopy(new_solution[0][new_customer_pos[0]][new_customer_pos[1]]))
            
            if ((t_1_x <= t and t_2_x < t) or (t_1_x < t and t_2_x <= t)):
                return local_search_min_time(instance, new_solution, n_nearest)

    for i in range(len(solution[0][1])):

        for customer in range(len(solution[0][1][i])):
            t_1 = 0 #analogo motivo di prima
            t_2 = drone_tour_time(instance.t_d, copy.deepcopy(solution[0][1][i]))
            t = max(t_1, t_2)

            new_solution = shift_d(instance, copy.deepcopy(solution), customer, i)
            new_customer_pos = get_position(customer, copy.deepcopy(solution[0])) #new_customer_pos contiene la pos del customer che è stato spostato dal drone tour al truck tour
            t_1_x = truck_tour_time(instance.t_t, copy.deepcopy(new_solution[0][new_customer_pos[0]][new_customer_pos[1]]))
            t_2_x = 0
            
            if ((t_1_x <= t and t_2_x < t) or (t_1_x < t and t_2_x <= t)):
                return local_search_min_time(instance, new_solution, n_nearest)

    return solution


In [288]:
def initial_solution_construction(instance, w1, w2, w3, w4, w5, gamma, n_nearest):
    A = [c for c in range (1, instance.N)]
    solution = recreate_min_time(instance, [[[[] for _ in range(instance.h)], [[] for _ in range(instance.D)]],A], w1, w2, w3, w4, w5, gamma)
    #solution = local_search_min_time(instance, solution, n_nearest)
    return solution

##### Third modification

In [351]:
def SISSRs_min_time(instance, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb):
    s_0 = initial_solution_construction(instance, w1, w2, w3, w4, w5, gamma, n_nearest)
    s_curr = s_0
    s_best = s_0
    '''iterations_without_improvement = 0
    iteration_counter = 0
    while (iteration_counter < iter_max):
        s = ruin_and_recreate(instance, copy.deepcopy(s_curr), sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma)
        if makespan(instance, s) < makespan(instance, s_curr)*(1+delta) or ((makespan(instance, s) == makespan(instance, s_curr) and total_completion_time(instance,s) < total_completion_time(instance, s_curr))):
            s_curr = local_search_min_time(instance, s)
            if makespan(instance, s_curr) < makespan(instance, s_best) or ((makespan(instance, s) == makespan(instance, s_best) and total_completion_time(instance,s) < total_completion_time(instance, s_best))):
                s_best = s_curr
                iterations_without_improvement = 0
            else:
                iterations_without_improvement += 1
        if iterations_without_improvement >= iter_max:
            s_curr = perturbate(instance, s_curr, p_min, p_max, max_unfeasible_swaps_perturb)
            iterations_without_improvement = 0
        delta = delta * epsilon
        iteration_counter+=1
'''
    return s_best

In [361]:
SISSRs_min_time(PDSVRP_instance.PDSVRPInstance("instances/30-c-0-c.txt"), 0.3, 4.5, 4.5, 5,1,1,2,2, 0.1, 20, 0.1, 0.999975, 100, 1000, 3, 3, 9)

feasible_truck_tour_positions:  [[0, 0, 0], [0, 1, 0], [0, 2, 0], [0, 3, 0], [0, 4, 0], [0, 5, 0], [0, 6, 0], [0, 7, 0], [0, 8, 0], [0, 9, 0]]
makespan:  0
makespan_diff_trucks:  0.4249465325844601
pos:  [0, 0, 0]
random value:  0.8029695081941037
I got here
makespan:  0
makespan_diff_trucks:  0.4249465325844601
pos:  [0, 1, 0]
random value:  0.8199111555958358
makespan:  0
makespan_diff_trucks:  0.4249465325844601
pos:  [0, 2, 0]
random value:  0.9675677244161611
I got here
makespan:  0
makespan_diff_trucks:  0.4249465325844601
pos:  [0, 3, 0]
random value:  0.41716931334804375
makespan:  0
makespan_diff_trucks:  0.4249465325844601
pos:  [0, 4, 0]
random value:  0.8922379503553499
makespan:  0
makespan_diff_trucks:  0.4249465325844601
pos:  [0, 5, 0]
random value:  0.5599725783354129
makespan:  0
makespan_diff_trucks:  0.4249465325844601
pos:  [0, 6, 0]
random value:  0.3600495736788881
makespan:  0
makespan_diff_trucks:  0.4249465325844601
pos:  [0, 7, 0]
random value:  0.15360530129

[[[[2, 1, 4], [3], [5], [], [], [], [], [], [], []], [[], [], [], []]], []]